# METOD algorithm - Shekel

# 1. Import libraries

The following libraries are required in order for the METOD Algorithm to run.

In [1]:
import numpy as np 
from numpy import linalg as LA
import pandas as pd

import metod_testing as mtv3

# 2. Define function and gradient

In [2]:
def f(x, d, m, matrix_test, C, B):
    total_sum = 0
    
    for i in range(m):
        
        sum_within_inverse = (x - C[:,i].reshape(d,)).T @  matrix_test[i] @ (x - C[:,i].reshape(d,))
        
            
        inverse_to_add = (sum_within_inverse + B[i]) ** (-1)
        total_sum += inverse_to_add
    return(-total_sum)

In [3]:
def g(x, d, m, matrix_test, C, B):
    init_grad = 0
    for i in range(m):
        numerator_1 = 2 * matrix_test[i] @ (x - C[:,i].reshape(d,)) 
        denom_act_1 = ((x - C[:,i].reshape(d,)).T @  matrix_test[i] @ (x - C[:,i].reshape(d,)) + B[i]) ** (2)
        init_grad += ((numerator_1 / denom_act_1))
    return init_grad.reshape(d,)

# 3. Defining parameters

In [5]:
P = 5
d = 50
proj = False
min_1_uniform = 0
min_2_uniform = 1
div = 1

lambda_1 = 1
lambda_2 = 10

In [6]:
C = np.zeros((d,P))
B = np.zeros((P))
A = np.zeros((P,d,d))
rotation = np.zeros((P,d,d))

for j in range(P):
    C[:,j] = np.random.uniform(min_1_uniform, min_2_uniform,(d)) 
    rotation[j] = mtv3.calculate_rotation_matrix(d, 3)
    B[j] = 1/div
    diag_vals = np.zeros(d)
    a = lambda_1
    diag_vals[0] = a
    b = lambda_2
    diag_vals[1] = b
    for i in range(2,d):
        diag_vals[i] = np.random.uniform(lambda_1 + 1, lambda_2 -1)
    A[j] = np.diag(diag_vals)

matrix_test = np.transpose(rotation, (0,2,1)) @ A @ rotation


We have:

•store_x0: $x_{0p}$ ($p=1,...,P$)

•matrix_combined: $A_p^T \Sigma_p A_p$ ($p=1,...,P$)


In [7]:
args = d, P, matrix_test, C, B

# 4. Run METOD Algorithm

Please see 'Multistart with early termination of  descents (METOD) algorithm for Python' for more information on how to choose appropiate parameters for the METOD Algorithm. 

In [8]:
discovered_minima, number_minima, func_vals_of_minima, excessive_no_descents  = mtv3.metod(f, g, args, d, beta=0.1, m=10)

100%|██████████| 999/999 [04:52<00:00,  3.32it/s]


# 5. Results of the METOD Algorithm

Total number of minima found:

In [9]:
number_minima

5

Positions of minima:

In [15]:
store_min = np.zeros((number_minima))
store_min_calc = np.zeros((number_minima))
index = 0
for el in discovered_minima:
    store_calc = np.zeros((number_minima))
    for j in range(number_minima):
        store_calc[j] = LA.norm(el - C[:,j])
        print(LA.norm(el - C[:,j]))
    store_min[index] = np.argmin(store_calc)
    store_min_calc[index] = np.min(store_calc)
    index += 1

2.6896356298397204
2.99434030073587
2.8651782269634576
0.006140634990371159
2.6897598728631666
2.8207508253511118
2.7483739821932294
3.06071641368647
2.689487360847728
0.006954406405623761
0.006077130960662879
2.5967389379048407
3.2151679511110656
2.6896427407047425
2.8212694277761337
2.5967843307851712
0.005974053498958345
2.7486353993393036
2.993965594380939
2.7491373260722867
3.214603742679398
2.7481167482554505
0.006297856549126373
2.8644296262694895
3.06047102597811


In [16]:
store_min 

array([3., 4., 0., 1., 2.])

In [17]:
store_min_calc

array([0.00614063, 0.00695441, 0.00607713, 0.00597405, 0.00629786])

Function values of minima:

In [13]:
func_vals_of_minima

[-1.0978655458931388,
 -1.098001039323246,
 -1.1003082035151912,
 -1.0996379172658024,
 -1.0825025777191073]

Total number of extra descents to an already discovered minima:

In [14]:
excessive_no_descents

33

# 6. Save results to csv file (optional)

The below csv files will be saved to the same folder which contains the METOD Algorithm - Minimum of several quadratic forms notebook.

Minima in msqf_discovered_minimas_d_%s_p_%s.csv are saved such that each row represents one discovered minima. The total number of rows will be the same as the value for number_minima.

In [ ]:
np.savetxt('msqf_discovered_minimas_d_%s_p_%s.csv' % (d, P), discovered_minima, delimiter=",")

Values in msqf_func_vals_discovered_minimas_d_%s_p_%s are saved such that each row represents the function value of a minima. The total number of rows will be the same as the value for number_minima.

In [ ]:
np.savetxt('msqf_func_vals_discovered_minimas_d_%s_p_%s.csv' % (d, P), func_vals_of_minima, delimiter=",")

msqf_summary_table_d_%s_p_%s.csv will contain the total number of minima discovered and the total number of extra descents.

In [ ]:
summary_table = pd.DataFrame({
"Total number of unique minima": [number_minima],
"Extra descents": [excessive_no_descents]})
summary_table.to_csv('msqf_summary_table_d_%s_p_%s.csv' % (d, P))

# Plot

In [ ]:
P = 3
d = 2
proj = False
rs = 1231
min_1_uniform = 0
min_2_uniform = 1
div = 2
projection = False
tolerance = 0.00001
option = 'minimize'
met = 'Nelder-Mead' 
initial_guess = 0.05
lambda_1 = 1
lambda_2 = 10

In [ ]:
C = np.zeros((d,P))
B = np.zeros((P))
A = np.zeros((P,d,d))
rotation = np.zeros((P,d,d))


np.random.seed(7376)
for j in range(P):
    #C[:,j] = np.random.uniform(min_1_uniform, min_2_uniform,(d)) 
    rotation[j] = mtv3.calculate_rotation_matrix(d, 1)
    B[j] = 1/div
    diag_vals = np.zeros(d)
    a = lambda_1
    diag_vals[0] = a
    b = lambda_2
    diag_vals[1] = b
    for i in range(2,d):
        diag_vals[i] = np.random.uniform(lambda_1 + 0.1, lambda_2 -0.1)
    A[j] = np.diag(diag_vals)

C[:,0] = np.array([0.8,0.8])
C[:,1] = np.array([0.2,0.2])
C[:,2] = np.array([0.9,0.1])
matrix_test = np.transpose(rotation, (0,2,1)) @ A @ rotation



In [ ]:
args = d, P, matrix_test, C, B

In [ ]:
import matplotlib.pyplot as plt
np.random.seed(9800)
test_num = 100
x = np.linspace(0, 1, test_num)
y = np.linspace(0, 1, test_num)
Z = np.zeros((test_num, test_num))
X, Y = np.meshgrid(x,y)
for i in range(test_num):
    for j in range(test_num):
        x1_var = X[i, j]
        x2_var = Y[i, j]
        x = np.array([x1_var, x2_var])
        Z[i, j] = f(x, *args)


for j in range(10):
    x = np.random.uniform(0,1,(d,))
    descended_x_points, its = mtv3.apply_sd_until_stopping_criteria(x, d, projection, tolerance, option, met, initial_guess, args, f, g)

    chosen_x1 = descended_x_points[0:descended_x_points.shape[0]][:,0]
    chosen_x2 = descended_x_points[0:descended_x_points.shape[0]][:,1]

    plt.scatter(chosen_x1, chosen_x2, s=20, color='blue')
    plt.plot(chosen_x1, chosen_x2, 'blue')

plt.contour(X, Y, Z, 50, cmap='RdGy', alpha=0.5)
#plt.savefig('shekel_d=2_rs_'+str(rs)+'.pdf')
plt.show()